# Import libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import pymysql
from sqlalchemy import create_engine

In [3]:
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler

In [4]:
# Import regression models

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [5]:
# Import scaling libraries

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [6]:
from sklearn.model_selection import cross_val_score

In [7]:
import warnings

# Dabase credentials

In [8]:
driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'cookies'

In [9]:
# Modifiyng numbers of columns displayed 
pd.set_option('display.max_columns',100)

# Create dataframe

In [10]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

query = 'SHOW TABLES'

user_df = pd.read_sql(query,engine)
user_df


query2 = """
SELECT * FROM cookies_quality
"""

cookies_df_original = pd.read_sql(query2, engine)

In [11]:
cookies_df = cookies_df_original.copy()

# Functions 

In [12]:
def graph_box_plots(df):
    '''
    Input: DataFrame
    
    # Create the boxplots for all the features of our dataset.
    '''
    cols = list(cookies_df.select_dtypes(include=['int64','float64']).columns)

    f, ax = plt.subplots(len(cols)//3,4, figsize=(12,10))

    plt.subplots_adjust(wspace=0.4,hspace=0.4)

    i = 0
    for row in ax:
        for col in row:
            if i == len(cols): 
                break
            else:    
                sns.boxplot(cookies_df[cols[i]], ax=col)
                i += 1

In [13]:
def clean_df(df):
    '''
    Input: DataFrame
    Output: DataFrame
    
    # Change crunch factor type, drop NaN values and drop 3 columns that don't give useful information.
    '''
    
    df['crunch factor'] = df['crunch factor'].astype('float')
    df = df.dropna()
    df = df.drop(['diameter','aesthetic appeal','id'], axis=1)
    
    return df

In [14]:
def encode(df):
    '''
    Input: DataFrame
    Outpu: DataFrame
    
    # One hot encodes categorical features. Drops columns that were encoded and one of the hot encoded
    # colums to avoid uniform information.
    '''
    
    flavour_list = ['raisins', 'nuts', 'chocolate', 'oats', 'peanut butter']
    
    for flavour in flavour_list:
        df[flavour] = 0

    for flavour in flavour_list:
        df[flavour] = np.where(df['mixins'].str.contains(flavour), 1, 0)
        
    
    df['butter_type_int'] = pd.get_dummies(df['butter type'],drop_first=True)
    
    df = df.drop(['mixins','butter type','raisins'],axis=1)
    
    return df

In [15]:
def cleanOutliers(df):
    '''
    Input: DataFrame
    Output: DataFrame
    
    # Returns a Dataframe without outliers
    '''
    
    cols = ['sugar to flour ratio', 'sugar index', 'bake temp', 'chill time',
       'calories', 'density', 'pH', 'grams baking soda', 'bake time',
       'quality', 'weight', 'crunch factor']
    
    for col in cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        df[col] = df[col].apply(lambda x: x if x > q1 - 3 * iqr and x < q3 + 3 * iqr else np.nan)   
        
    df = df.dropna()
    
    return df

In [16]:
def conversion(x):
    '''
    Input: Series
    
    # Maps values of a given list, array or Series.
    '''
    
    if x >= 9:
        return 2
    elif x <= 6:
        return 0
    else: 
        return 1

In [17]:
def convert_quality(df):
    '''
    Input: DataFrame
    
    # Maps values of quality Series with the conversion function.
    '''
    
    df['quality_label'] = df['quality'].apply(lambda x: conversion(x))

In [18]:
def prepare_for_scaling(df):
    '''
    Input: DataFrame
    Output: DataFrame
    
    # Return a DataFrame without the target and hot encoded columns.
    '''
    column_list = ['quality','nuts', 'chocolate', 'oats', 'peanut butter','butter_type_int']
    
    return df.drop(column_list, axis=1)

In [19]:
def normalizer(df):
    '''
    Input: DataFrame
    Output: Array
    
    # Normalize samples individually to unit norm.
    '''
    df_normal = prepare_for_scaling(df)
    transformer = Normalizer().fit(df_normal)
    
    return transformer.transform(df_normal)

In [20]:
def robust_scaler(df):
    '''
    Input: DataFrame
    Output: Array
    
    # Scale features using statistics that are robust to outliers.
    '''
    df_robust = prepare_for_scaling(df)
    transformer = RobustScaler().fit(df_robust)
    
    return transformer.transform(df_robust)

In [21]:
def min_max_scaler(df):
    '''
    Input: DataFrame
    Output: DataFrame
    
    # Transforms features by scaling each feature to a given range.
    '''
    df = prepare_for_scaling(df)
    transformer = MinMaxScaler().fit(df)
    
    return transformer.transform(df)

In [22]:
def standard_scaler(df):
    '''
    Input: DataFrame
    Output: Array
    
    # Standardize features by removing the mean and scaling to unit variance.
    '''
    df = prepare_for_scaling(df)
    scaler = StandardScaler().fit(df)
    
    return scaler.transform(df)

In [23]:
def create_definitive_df(array, df):
    '''
    Input: Array, DataFrame
    Output: DataFrame
    
    # Converts the normalized/standardized array to a DataFrame and concats the result
    # to the one hot encoded and quality columns.
    '''
    
    columns_set = set(df.columns)
    column_list = ['quality', 'nuts', 'chocolate', 'oats', 'peanut butter','butter_type_int']
    normalized_columns = columns_set.difference(set(column_list))
    normalized_df = pd.DataFrame(array, columns=normalized_columns)
    concat_df = pd.concat([normalized_df, df[column_list].reset_index(drop=True)], axis=1,ignore_index=False)
    
    return concat_df

# Creating models

In [24]:
lin_model = LinearRegression()
knn_model = KNeighborsRegressor()
tree_model = DecisionTreeRegressor()
forest_model = RandomForestRegressor()
svr_model = SVR()

# List with the different models that will be used

In [25]:
models = [lin_model,knn_model,tree_model,forest_model,svr_model]

In [26]:
# Ignoring possible errors, when we use cross validation mostly.
warnings.filterwarnings('ignore')

# Trying different models

## TRY NUMBER 1

In [27]:
# With outliers and scaling = normalizer

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_encoded = encode(cookies_cleaned)
cookies_normalized = normalizer(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)
X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_1 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_1.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_1)):
    print(f'{models_names[i]}: {scores_list_try_1[i][1]}')

Linear Regression: 0.6335129333289574
KNN: 0.6366434279438353
DecisionTree: 0.5187033540642194
RandomForest: 0.7204303182507275
SVR: 0.5783735573233884


## TRY NUMBER 2

In [28]:
# Without outliers and scaling = normalizer

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_notOL = cleanOutliers(cookies_cleaned)
cookies_encoded = encode(cookies_notOL)
cookies_normalized = normalizer(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)
# cookies_concat.dropna(inplace=True)

X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_2 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_2.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_2)):
    print(f'{models_names[i]}: {scores_list_try_2[i][1]}')

Linear Regression: 0.6529612304311
KNN: 0.6198366532500843
DecisionTree: 0.532324402335155
RandomForest: 0.7169196941088928
SVR: 0.5556063764934189


## TRY NUMBER 3

In [29]:
# Without outliers and scaling = robust

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_notOL = cleanOutliers(cookies_cleaned)
cookies_encoded = encode(cookies_notOL)
cookies_normalized = robust_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)
# cookies_concat.dropna(inplace=True)

X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_3 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_3.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_3)):
    print(f'{models_names[i]}: {scores_list_try_3[i][1]}')

Linear Regression: 0.6637698413181574
KNN: 0.6774788969734425
DecisionTree: 0.5544243306946776
RandomForest: 0.7411399653731918
SVR: 0.7054696384586268


## TRY NUMBER 4 (BEST TRY)

In [30]:
# Without outliers and scaling = minmaxscaler()

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_notOL = cleanOutliers(cookies_cleaned)
cookies_encoded = encode(cookies_notOL)
cookies_normalized = min_max_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)

X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_4 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_4.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_4)):
    print(f'{models_names[i]}: {scores_list_try_4[i][1]}')

Linear Regression: 0.6637698413181576
KNN: 0.6581967184191442
DecisionTree: 0.5549282086490248
RandomForest: 0.740697233870638
SVR: 0.6746884063377131


## TRY NUMBER 5

In [31]:
# Without outliers and scaling = minmaxscaler()

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_notOL = cleanOutliers(cookies_cleaned)
cookies_encoded = encode(cookies_notOL)
cookies_normalized = standard_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)

X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_5 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_5.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_5)):
    print(f'{models_names[i]}: {scores_list_try_5[i][1]}')

Linear Regression: 0.6637698413181575
KNN: 0.6692749707446497
DecisionTree: 0.5589976526883833
RandomForest: 0.7385827517663344
SVR: 0.7076840695098625


## TRY NUMBER 6

In [32]:
# With outliers and scaling = robust

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_encoded = encode(cookies_cleaned)
cookies_normalized = robust_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)
X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_6 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_6.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_6)):
    print(f'{models_names[i]}: {scores_list_try_6[i][1]}')

Linear Regression: -1.72786555751775
KNN: 0.6847597051630185
DecisionTree: 0.5558962960421855
RandomForest: 0.7426520853459688
SVR: 0.7151496303787646


## TRY NUMBER 7

In [33]:
# With outliers and scaling = standardscaler

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_encoded = encode(cookies_cleaned)
cookies_normalized = standard_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)
X = cookies_concat.drop('quality',axis=1)
y = cookies_concat['quality']

scores_list_try_7 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_7.append((i,np.mean(cv)))
    
for i in range(len(scores_list_try_7)):
    print(f'{models_names[i]}: {scores_list_try_7[i][1]}')

Linear Regression: -1.7278655575177513
KNN: 0.6701815517499424
DecisionTree: 0.5557551033631674
RandomForest: 0.7429795972250524
SVR: 0.7135625789010606


## TRY NUMBER 8

In [34]:
# With outliers and scaling = standardscaler

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_notOL = cleanOutliers(cookies_cleaned)
cookies_encoded = encode(cookies_notOL)
cookies_normalized = min_max_scaler(cookies_encoded)

cookies_concat_pref = create_definitive_df(cookies_normalized,cookies_encoded)
cookies_concat = convert_quality(cookies_concat_pref)

X = cookies_concat.drop(['quality','quality_label'],axis=1)
y = cookies_concat['quality_label']

scores_list_try_8 = []

models_names = ['Linear Regression','KNN','DecisionTree','RandomForest','SVR']

for i, model in enumerate(models):
    cv = cross_val_score(models[i],X,y,cv=10,scoring='r2')
    scores_list_try_8.append((i,np.mean(cv)))

for i in range(len(scores_list_try_8)):
    print(f'{models_names[i]}: {scores_list_try_8[i][1]}')

AttributeError: 'NoneType' object has no attribute 'drop'

# Test

In [43]:
query_test = """
SELECT * FROM cookies_evaluate
"""

cookies_df_test = pd.read_sql(query_test, engine)

In [44]:
cookies_df_test.head()

,id,sugar to flour ratio,sugar index,bake temp,chill time,calories,density,pH,grams baking soda,bake time,quality,butter type,weight,diameter,mixins,crunch factor,aesthetic appeal
0,1,0.35,1.0,520,35.0,146.0,0.9930,8.45,0.44,10.0,0,melted,13.8,7,chocolate,1.43,3
1,2,0.39,10.4,440,20.0,142.0,0.9974,8.20,0.53,10.0,0,melted,17.0,7,chocolate,1.57,3
2,3,0.33,1.1,570,21.0,82.0,0.9910,8.32,0.46,10.9,0,melted,12.4,7,"chocolate, oats",1.44,3
3,4,0.37,13.5,600,52.0,192.0,0.9975,8.00,0.44,9.1,0,melted,14.8,7,chocolate,1.51,3
4,5,0.31,5.5,480,42.0,173.0,0.9951,8.19,0.66,9.3,0,melted,12.8,7,raisins,1.86,3


In [45]:
# Clean train data set

cookies_df = cookies_df_original.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_notOL = cleanOutliers(cookies_cleaned)
cookies_encoded = encode(cookies_notOL)
cookies_normalized = min_max_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)

X_train = cookies_concat.drop('quality',axis=1)
y_train = cookies_concat['quality']

In [48]:
# Clean test data set

cookies_df = cookies_df_test.copy()

cookies_cleaned = clean_df(cookies_df)
cookies_encoded = encode(cookies_cleaned)
cookies_normalized = min_max_scaler(cookies_encoded)

cookies_concat = create_definitive_df(cookies_normalized,cookies_encoded)

X_test = cookies_concat.drop('quality',axis=1)
y_test = cookies_concat['quality']


In [49]:
rf = RandomForestRegressor()

In [50]:
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [51]:
preds = rf.predict(X_test)

In [52]:
cookies_concat['predictions'] = preds

In [54]:
cookies_concat.head()

,chill time,weight,sugar index,crunch factor,calories,bake time,bake temp,density,grams baking soda,pH,sugar to flour ratio,quality,nuts,chocolate,oats,peanut butter,butter_type_int,predictions
0,0.35,0.006135,0.198020,0.114983,0.319444,0.107531,0.680412,0.221053,0.263158,0.241758,0.43,0,0,1,0,0,1,7.7
1,0.39,0.150307,0.158416,0.062718,0.310185,0.192935,0.422680,0.315789,0.263158,0.417582,0.57,0,0,1,0,0,1,7.8
2,0.33,0.007669,0.222772,0.066202,0.171296,0.068711,0.546392,0.242105,0.421053,0.164835,0.44,0,0,1,1,0,1,7.3
3,0.37,0.197853,0.237624,0.174216,0.425926,0.194876,0.216495,0.221053,0.105263,0.296703,0.51,0,0,1,0,0,1,7.6
4,0.31,0.075153,0.178218,0.139373,0.381944,0.148292,0.412371,0.452632,0.140351,0.186813,0.86,0,0,0,0,0,1,7.7


In [56]:
cookies_concat.to_csv('cookies_prediction_gameofcookies.csv', index=False)